# Proyecto Open Data I
## Radares, y su eficiencia en la CAM
### Recopilación, limpieza y almacenamiento de los datos
Este cuaderno pretende enseñar el proceso de limpieza de los datos relativos a los radares en la CAM
_Paula Gómez Lucas_

In [982]:
# Importar librerías
import os
import pandas as pd
import numpy as np
import math
from pyproj import Proj, transform

A continuación, se muestra la clase que está compuesta de todos los métodos que se encargan de la limpieza y transformación de los datos

In [983]:
ds = {}
folders = ("datasets/actuacionesBomberos", "datasets/estaciones", "datasets/accidentalidad")
folders

('datasets/actuacionesBomberos',
 'datasets/estaciones',
 'datasets/accidentalidad')

In [984]:
for folder in folders:
    df = pd.DataFrame()
    filename = folder[9:]

    for file in os.listdir(folder):
            filepath = folder + "/" + file
            if folder == "datasets/actuacionesBomberos":
                df1 = pd.read_csv(filepath, sep=';', decimal=',', encoding='iso-8859-1', low_memory=False)
            else:
                df1 = pd.read_csv(filepath, sep=';', decimal=',', encoding='utf-8', low_memory=False)
            df = pd.concat([df, df1])

    ds[filename] = df

In [985]:
folder_path = "datasets"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
csv_files

['calendario.csv',
 'DireccionesVigentes_20240204.csv',
 'iluminacion.csv',
 'padron22.csv',
 'radares.csv',
 'unidades_luminosas_m30.csv']

In [986]:
ds['calendario'] = pd.read_csv('datasets/calendario.csv', sep=',', encoding='iso-8859-1', low_memory=False)
ds['direcciones'] = pd.read_csv('datasets/DireccionesVigentes_20240204.csv', sep=',', encoding='utf-8-sig', low_memory=False, decimal=".")
ds['iluminacion'] = pd.read_csv('datasets/iluminacion.csv', sep=',', encoding='utf-8-sig', low_memory=False, decimal=".")
ds['padron'] = pd.read_csv('datasets/padron22.csv', sep=',', encoding='iso-8859-1', low_memory=False)
ds['radares'] = pd.read_csv('datasets/radares.csv', sep=',', encoding='utf-8-sig', low_memory=False, decimal=".")
iluminacion = pd.read_csv('datasets/unidades_luminosas_m30.csv', sep=';', encoding='utf-8-sig', low_memory=False, decimal=',')
iluminacion.columns

Index(['tipo_bloque,C,50', 'TIPO,C,254', 'VIA_CLASE,C,254', 'VIA_PAR,C,254',
       'VIA_NOMBRE,C,254', 'CLASE_APP,C,254', 'NUMERO,N,10,0',
       'COD_NDP,N,10,0', 'DISTRITO,N,10,0', 'BARRIO,N,10,0', 'X_UTM,N,19,11',
       'Y_UTM,N,19,11'],
      dtype='object')

In [987]:
iluminacion.rename(columns={'tipo_bloque,C,50':'tipo_bloque', 'TIPO,C,254':'TIPO', 'VIA_CLASE,C,254':'VIA_CLASE', 'VIA_PAR,C,254':'VIA_PAR', 'VIA_NOMBRE,C,254':'VIA_NOMBRE', 'CLASE_APP,C,254':'CLASE_APP', 'NUMERO,N,10,0':'NUMERO', 'COD_NDP,N,10,0':'COD_NDP', 'DISTRITO,N,10,0':'DISTRITO', 'BARRIO,N,10,0':'BARRIO', 'X_UTM,N,19,11':'X_UTM', 'Y_UTM,N,19,11':'Y_UTM'}, inplace=True) 
print(iluminacion.columns)
print(ds['iluminacion'].columns)

Index(['tipo_bloque', 'TIPO', 'VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE',
       'CLASE_APP', 'NUMERO', 'COD_NDP', 'DISTRITO', 'BARRIO', 'X_UTM',
       'Y_UTM'],
      dtype='object')
Index(['tipo_bloque', 'TIPO', 'VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE',
       'CLASE_APP', 'NUMERO', 'COD_NDP', 'DISTRITO', 'BARRIO', 'X_UTM',
       'Y_UTM'],
      dtype='object')


In [988]:
ds['iluminacion'] = pd.concat([ds['iluminacion'], iluminacion])
ds['iluminacion']

,tipo_bloque,TIPO,VIA_CLASE,VIA_PAR,VIA_NOMBRE,CLASE_APP,NUMERO,COD_NDP,DISTRITO,BARRIO,X_UTM,Y_UTM
0,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,3,20040016,16,4,445648.4812,4480670.67
1,LBE002,DESCARGA,CALLE,DE LAS,ERAS,NUMERO,10,31024246,16,4,445610.6416,4480740.996
2,GLED001,LED,PLAZA,DE,MAR DEL PLATA,NUMERO,12,11119324,16,4,445680.0750,4480679.164
3,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,21,11119327,16,4,445669.5353,4480644.66
4,FFLED005,LED,CALLE,DEL,MAR AMARILLO,NUMERO,19,11119326,16,4,445651.1627,4480635.018
...,...,...,...,...,...,...,...,...,...,...,...,...
3609,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+370,99WT37ALUM01,9,1,437437.9738,"4476224,266,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3610,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+420,99WT42ALUM01,9,1,437493.3321,"4476217,74,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3611,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+460,99WT46ALUM01,9,1,437532.6933,"4476210,583,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
3612,B - Báculo.,SA - Sodio alta presión.,AUTOVIA,NaN,M-500,KM,99+500,99WT50ALUM01,9,1,437571.4231,"4476206,794,,,,,,,,,,,,,,,,,,,,,,,,,,,,"


Observamos que ambos datasets son un único dataframe ahora, teniendo todas las unidades luminosas en uno único. A continuación, eliminamos las variables auxiliares que hemos utilizado.

In [989]:
del iluminacion
del df
del df1

Limpiamos los datos eliminando las columnas autogeneradas con NaNs, renombramos las columnas para que sean uniformes (minúsculas y con barra bajas), eliminamos las filas de NaNs, eliminamos las filas duplicadas.

In [990]:
columna_borrar = "Unnamed"
for df in ds:
    for j in ds[df].columns:
        if columna_borrar in j:
            while j in ds[df].columns:
                ds[df].drop(j, axis=1, inplace=True)
                ds[df].dropna(how='all', axis=0, inplace=True)

    ds[df].rename(columns = lambda x: x.strip().lower().replace(' ', '_'), inplace=True)
    ds[df] = ds[df].map(lambda x: x.strip() if isinstance(x, str) else x)
    ds[df].dropna(how='all', axis=0, inplace=True)
    ds[df].drop_duplicates(inplace=True)
    ds[df] = ds[df].loc[:, ~ds[df].columns.duplicated()]

Para confirmar que los datos se van limpiando bien, utilizaremos el método get_info para ver cuántos datos en cada columna quedan nulos, así como usamos el método info() de pandas para ver un resumen de todas las columnas y comprobamos también que todo el formateo de las columnas se ha realizado sin problema.

In [991]:
def get_info(filename):
    print(ds[filename].isnull().sum())
    print(ds[filename].info())

El siguiente paso es el análisis de los datasets columna a columna para revisar qué método usar para rellenar los datos faltantes dependiendo de cada atributo.  

### Sustitución de valores faltantes
El dataset estaciones no tienen datos faltantes, por lo que sólo nos queda trabajar con los otros 8 datasets, que podemos observar aquí:

In [992]:
ds.keys()

dict_keys(['actuacionesBomberos', 'estaciones', 'accidentalidad', 'calendario', 'direcciones', 'iluminacion', 'padron', 'radares'])

#### Actuaciones de bomberos

In [993]:
get_info('actuacionesBomberos')

año                         1043
mes                            1
distrito                       1
fuegos                         1
daños_en_construccion       1043
salvamentos_y_rescates         1
daños_por_agua              1043
incidentes_diversos            1
salidas_sin_intervencion       1
servicios_varios               1
total                          1
ï»¿aão                      259
daãos_en_construccion       260
daãos_por_agua              260
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       259 non-null    float64
 1   mes                       1301 non-null   object 
 2   distrito                  1301 non-null   object 
 3   fuegos                    1301 non-null   float64
 4   daños_en_construccion     259 non-null    float64
 5   salvamentos_y_rescates    1301 non

In [994]:
str(ds['actuacionesBomberos'].columns)

"Index(['año', 'mes', 'distrito', 'fuegos', 'daños_en_construccion',\n       'salvamentos_y_rescates', 'daños_por_agua', 'incidentes_diversos',\n       'salidas_sin_intervencion', 'servicios_varios', 'total', 'ï»¿aã\x91o',\n       'daã\x91os_en_construccion', 'daã\x91os_por_agua'],\n      dtype='object')"

Vemos que las columnas de daños por agua, construcción y año, se han disgregado en alguno de los csv iniciales por formato, por lo que salen como faltantes valores que realmente no lo son. Una solución es sustituir los faltantes por 0 y sumar los valores de las columnas a mergear porque así se concatenan los valores sin despreciar su tipo.

In [995]:
ds['actuacionesBomberos']['daños_por_agua'] = ds['actuacionesBomberos']['daños_por_agua'].fillna(0)
ds['actuacionesBomberos']['daños_en_construccion'] = ds['actuacionesBomberos']['daños_en_construccion'].fillna(0)
ds['actuacionesBomberos']['año'] = ds['actuacionesBomberos']['año'].fillna(0)
ds['actuacionesBomberos']['ï»¿aã\x91o'] = ds['actuacionesBomberos']['ï»¿aã\x91o'].fillna(0)
ds['actuacionesBomberos']['daã\x91os_en_construccion'] = ds['actuacionesBomberos']['daã\x91os_en_construccion'].fillna(0)
ds['actuacionesBomberos']['daã\x91os_por_agua'] = ds['actuacionesBomberos']['daã\x91os_por_agua'].fillna(0)


In [996]:
get_info('actuacionesBomberos')
ds['actuacionesBomberos']

año                         0
mes                         1
distrito                    1
fuegos                      1
daños_en_construccion       0
salvamentos_y_rescates      1
daños_por_agua              0
incidentes_diversos         1
salidas_sin_intervencion    1
servicios_varios            1
total                       1
ï»¿aão                     0
daãos_en_construccion      0
daãos_por_agua             0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       1302 non-null   float64
 1   mes                       1301 non-null   object 
 2   distrito                  1301 non-null   object 
 3   fuegos                    1301 non-null   float64
 4   daños_en_construccion     1302 non-null   float64
 5   salvamentos_y_rescates    1301 non-null   float64
 6   daños_por_agua       

,año,mes,distrito,fuegos,daños_en_construccion,salvamentos_y_rescates,daños_por_agua,incidentes_diversos,salidas_sin_intervencion,servicios_varios,total,ï»¿aão,daãos_en_construccion,daãos_por_agua
0,2019.0,Enero,CENTRO,27.0,20.0,35.0,21.0,41.0,16.0,10.0,170.0,0.0,0.0,0.0
1,2019.0,Enero,ARGANZUELA,16.0,9.0,13.0,8.0,24.0,8.0,3.0,81.0,0.0,0.0,0.0
2,2019.0,Enero,RETIRO,7.0,4.0,7.0,4.0,18.0,6.0,4.0,50.0,0.0,0.0,0.0
3,2019.0,Enero,SALAMANCA,21.0,15.0,16.0,14.0,31.0,17.0,7.0,121.0,0.0,0.0,0.0
4,2019.0,Enero,CHAMARTIN,12.0,7.0,21.0,5.0,25.0,6.0,5.0,81.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,0.0,enero,VILLA DE VALLECAS,15.0,0.0,19.0,0.0,7.0,22.0,2.0,77.0,2023.0,7.0,5.0
260,0.0,enero,VICALVARO,9.0,0.0,8.0,0.0,8.0,6.0,2.0,41.0,2023.0,5.0,3.0
261,0.0,enero,SAN BLAS,16.0,0.0,25.0,0.0,10.0,20.0,1.0,79.0,2023.0,3.0,4.0
262,0.0,enero,BARAJAS,1.0,0.0,4.0,0.0,15.0,13.0,8.0,45.0,2023.0,2.0,2.0


In [997]:
ds['actuacionesBomberos']['daños_por_agua'] = ds['actuacionesBomberos']['daños_por_agua'] + ds['actuacionesBomberos']['daã\x91os_por_agua']
ds['actuacionesBomberos']['daños_en_construccion'] = ds['actuacionesBomberos']['daños_en_construccion'] + ds['actuacionesBomberos']['daã\x91os_en_construccion']
ds['actuacionesBomberos']['año'] = ds['actuacionesBomberos']['año'] + ds['actuacionesBomberos']['ï»¿aã\x91o']

In [998]:
ds['actuacionesBomberos'].drop('ï»¿aã\x91o', axis=1, inplace=True)
ds['actuacionesBomberos'].drop('daã\x91os_por_agua', axis=1, inplace=True)
ds['actuacionesBomberos'].drop('daã\x91os_en_construccion', axis=1, inplace=True)

In [999]:
get_info('actuacionesBomberos')
ds['actuacionesBomberos']

año                         0
mes                         1
distrito                    1
fuegos                      1
daños_en_construccion       0
salvamentos_y_rescates      1
daños_por_agua              0
incidentes_diversos         1
salidas_sin_intervencion    1
servicios_varios            1
total                       1
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 263
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       1302 non-null   float64
 1   mes                       1301 non-null   object 
 2   distrito                  1301 non-null   object 
 3   fuegos                    1301 non-null   float64
 4   daños_en_construccion     1302 non-null   float64
 5   salvamentos_y_rescates    1301 non-null   float64
 6   daños_por_agua            1302 non-null   float64
 7   incidentes_diversos       1301 non-null   float64
 8   s

,año,mes,distrito,fuegos,daños_en_construccion,salvamentos_y_rescates,daños_por_agua,incidentes_diversos,salidas_sin_intervencion,servicios_varios,total
0,2019.0,Enero,CENTRO,27.0,20.0,35.0,21.0,41.0,16.0,10.0,170.0
1,2019.0,Enero,ARGANZUELA,16.0,9.0,13.0,8.0,24.0,8.0,3.0,81.0
2,2019.0,Enero,RETIRO,7.0,4.0,7.0,4.0,18.0,6.0,4.0,50.0
3,2019.0,Enero,SALAMANCA,21.0,15.0,16.0,14.0,31.0,17.0,7.0,121.0
4,2019.0,Enero,CHAMARTIN,12.0,7.0,21.0,5.0,25.0,6.0,5.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...
259,2023.0,enero,VILLA DE VALLECAS,15.0,7.0,19.0,5.0,7.0,22.0,2.0,77.0
260,2023.0,enero,VICALVARO,9.0,5.0,8.0,3.0,8.0,6.0,2.0,41.0
261,2023.0,enero,SAN BLAS,16.0,3.0,25.0,4.0,10.0,20.0,1.0,79.0
262,2023.0,enero,BARAJAS,1.0,2.0,4.0,2.0,15.0,13.0,8.0,45.0


Debido a que el MV que encontramos está en todas las columnas menos en las que sustituimos por 0 para realizar la limpieza, parece que todos los valores realmente eran faltantes inicialmente: 

In [1000]:
ds['actuacionesBomberos'][ds['actuacionesBomberos'].isna().any(axis=1)]

,año,mes,distrito,fuegos,daños_en_construccion,salvamentos_y_rescates,daños_por_agua,incidentes_diversos,salidas_sin_intervencion,servicios_varios,total
257,2020.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN


In [1001]:
ds['actuacionesBomberos'].drop(257, axis=0, inplace=True)

In [1002]:
ds['actuacionesBomberos']['año'] = ds['actuacionesBomberos']['año'].astype('int64')
ds['actuacionesBomberos']['fuegos'] = ds['actuacionesBomberos']['fuegos'].astype('int64')
ds['actuacionesBomberos']['daños_en_construccion'] = ds['actuacionesBomberos']['daños_en_construccion'].astype('int64')
ds['actuacionesBomberos']['salvamentos_y_rescates'] = ds['actuacionesBomberos']['salvamentos_y_rescates'].astype('int64')
ds['actuacionesBomberos']['daños_por_agua'] = ds['actuacionesBomberos']['daños_por_agua'].astype('int64')
ds['actuacionesBomberos']['incidentes_diversos'] = ds['actuacionesBomberos']['incidentes_diversos'].astype('int64')
ds['actuacionesBomberos']['salidas_sin_intervencion'] = ds['actuacionesBomberos']['salidas_sin_intervencion'].astype('int64')
ds['actuacionesBomberos']['servicios_varios'] = ds['actuacionesBomberos']['servicios_varios'].astype('int64')
ds['actuacionesBomberos']['total'] = ds['actuacionesBomberos']['total'].astype('int64')

In [1003]:
ds['actuacionesBomberos']['mes'] = ds['actuacionesBomberos']['mes'].str.lower()
meses = {
    1:'enero',
    2:'febrero',
    3:'marzo',
    4:'abril',
    5:'mayo',
    6:'junio',
    7:'julio',
    8:'agosto',
    9:'septiembre',
    10:'octubre',
    11:'noviembre',
    12:'diciembre'
}

ds['actuacionesBomberos']['idMes'] = ds['actuacionesBomberos']['mes'].map({v: k for k, v in meses.items()})

In [1004]:
get_info('actuacionesBomberos')
ds['actuacionesBomberos']

año                         0
mes                         0
distrito                    0
fuegos                      0
daños_en_construccion       0
salvamentos_y_rescates      0
daños_por_agua              0
incidentes_diversos         0
salidas_sin_intervencion    0
servicios_varios            0
total                       0
idMes                       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 1297 entries, 0 to 263
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   año                       1297 non-null   int64 
 1   mes                       1297 non-null   object
 2   distrito                  1297 non-null   object
 3   fuegos                    1297 non-null   int64 
 4   daños_en_construccion     1297 non-null   int64 
 5   salvamentos_y_rescates    1297 non-null   int64 
 6   daños_por_agua            1297 non-null   int64 
 7   incidentes_diversos       1297 non-

,año,mes,distrito,fuegos,daños_en_construccion,salvamentos_y_rescates,daños_por_agua,incidentes_diversos,salidas_sin_intervencion,servicios_varios,total,idMes
0,2019,enero,CENTRO,27,20,35,21,41,16,10,170,1
1,2019,enero,ARGANZUELA,16,9,13,8,24,8,3,81,1
2,2019,enero,RETIRO,7,4,7,4,18,6,4,50,1
3,2019,enero,SALAMANCA,21,15,16,14,31,17,7,121,1
4,2019,enero,CHAMARTIN,12,7,21,5,25,6,5,81,1
...,...,...,...,...,...,...,...,...,...,...,...,...
259,2023,enero,VILLA DE VALLECAS,15,7,19,5,7,22,2,77,1
260,2023,enero,VICALVARO,9,5,8,3,8,6,2,41,1
261,2023,enero,SAN BLAS,16,3,25,4,10,20,1,79,1
262,2023,enero,BARAJAS,1,2,4,2,15,13,8,45,1


#### Accidentalidad

In [1005]:
get_info('accidentalidad')
ds['accidentalidad']

num_expediente               0
fecha                        0
hora                         0
localizacion                 0
numero                       6
cod_distrito                 6
distrito                     6
tipo_accidente               5
estado_meteorológico     20120
tipo_vehiculo              816
tipo_persona                 3
rango_edad                   0
sexo                         0
cod_lesividad            85278
lesividad                85278
coordenada_x_utm             3
coordenada_y_utm             3
positiva_alcohol           718
positiva_droga          195001
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 31157
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   num_expediente        195610 non-null  object 
 1   fecha                 195610 non-null  object 
 2   hora                  195610 non-null  object 
 3   localizacion          195610

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2018S017842,04/02/2019,9:10:00,"CALL. ALBERTO AGUILERA, 1",1,1.0,CENTRO,Colisión lateral,Despejado,Motocicleta > 125cc,Conductor,De 45 a 49 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,"440068,049","4475679,17",N,NaN
1,2018S017842,04/02/2019,9:10:00,"CALL. ALBERTO AGUILERA, 1",1,1.0,CENTRO,Colisión lateral,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,"440068,049","4475679,17",N,NaN
2,2019S000001,01/01/2019,3:45:00,PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA,168,11.0,CARABANCHEL,Alcance,NaN,Furgoneta,Conductor,De 40 a 44 años,Hombre,NaN,NaN,"439139,603","4470836,854",S,NaN
3,2019S000001,01/01/2019,3:45:00,PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA,168,11.0,CARABANCHEL,Alcance,NaN,Turismo,Conductor,De 40 a 44 años,Mujer,NaN,NaN,"439139,603","4470836,854",N,NaN
4,2019S000001,01/01/2019,3:45:00,PASEO. SANTA MARIA DE LA CABEZA / PLAZA. ELIPTICA,168,11.0,CARABANCHEL,Alcance,NaN,Turismo,Conductor,De 45 a 49 años,Mujer,NaN,NaN,"439139,603","4470836,854",N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31153,2023S028337,31/08/2023,23:20:00,"CALL. GOLFO DE SALONICA, 12",12,16.0,HORTALEZA,Otro,Despejado,Turismo,Pasajero,De 55 a 59 años,Mujer,14.0,Sin asistencia sanitaria,443580.365,4480912.376,N,NaN
31154,2023S028341,31/08/2023,22:30:00,CALLE SAN SERAPIO,1,12.0,USERA,Alcance,Despejado,Motocicleta hasta 125cc,Conductor,De 30 a 34 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,440795.550,4471044.789,N,NaN
31155,2023S028341,31/08/2023,22:30:00,CALLE SAN SERAPIO,1,12.0,USERA,Alcance,Despejado,Turismo,Conductor,De 35 a 39 años,Hombre,14.0,Sin asistencia sanitaria,440795.550,4471044.789,N,NaN
31156,2023S028352,31/08/2023,14:40:00,"AUTOV. M-30, 00NC06",00NC06,5.0,CHAMARTÍN,Caída,Despejado,Motocicleta hasta 125cc,Conductor,De 40 a 44 años,Hombre,1.0,Atención en urgencias sin posterior ingreso,442825.500,4481003.193,N,NaN


In [1006]:
lesividadDict = ds['accidentalidad'].set_index('cod_lesividad')['lesividad'].to_dict()
lesividadDict = {int(k): v for k, v in lesividadDict.items() if not math.isnan(k)}
lesividadDict[14] = 'Sin asistencia sanitaria'
lesividadDict


{7: 'Asistencia sanitaria sólo en el lugar del accidente',
 2: 'Ingreso inferior o igual a 24 horas',
 14: 'Sin asistencia sanitaria',
 5: 'Asistencia sanitaria ambulatoria con posterioridad',
 3: 'Ingreso superior a 24 horas',
 1: 'Atención en urgencias sin posterior ingreso',
 6: 'Asistencia sanitaria inmediata en centro de salud o mutua',
 4: 'Fallecido 24 horas',
 77: 'Se desconoce'}

Hay algunos datos faltantes que tiene sentido que lo sean, y podemos sustituir por un buzzword de algún tipo que nos haga saber que se trate de esto, como lo es que en un accidente en el que no se han producido lesiones, la lesividad sea nula y tampoco haya código de la misma, y como ambas cifras coinciden, es lógico pensar que se trata de las mismas situaciones. Podemos sustituir entonces todos los NaNs de lesividad faltantes por 'Sin asistencia sanitaria' y el código por 14, ya que si no hay registro, es que no se realizó dicha asistencia.

In [1007]:
ds['accidentalidad']['cod_lesividad'] = ds['accidentalidad']['cod_lesividad'].fillna(14.0)
ds['accidentalidad']['lesividad'] = ds['accidentalidad']['lesividad'].fillna(ds['accidentalidad']['cod_lesividad'].map(lesividadDict))

In [1008]:
ds['accidentalidad']['lesividad'] = np.where(ds['accidentalidad']['cod_lesividad'] == 14,
                                             ds['accidentalidad']['lesividad'].fillna('Sin asistencia sanitaria'),
                                             ds['accidentalidad']['lesividad'])


Ocurre una situación similar en distrito y código distrito

In [1009]:
distritoDict = ds['accidentalidad'].set_index('cod_distrito')['distrito'].to_dict()
distritoDict = {int(k): v for k, v in distritoDict.items() if not math.isnan(k)}
distritoDict

{1: 'CENTRO',
 11: 'CARABANCHEL',
 10: 'LATINA',
 12: 'USERA',
 9: 'MONCLOA-ARAVACA',
 14: 'MORATALAZ',
 4: 'SALAMANCA',
 18: 'VILLA DE VALLECAS',
 17: 'VILLAVERDE',
 7: 'CHAMBERÍ',
 5: 'CHAMARTÍN',
 16: 'HORTALEZA',
 15: 'CIUDAD LINEAL',
 3: 'RETIRO',
 8: 'FUENCARRAL-EL PARDO',
 19: 'VICÁLVARO',
 13: 'PUENTE DE VALLECAS',
 21: 'BARAJAS',
 2: 'ARGANZUELA',
 6: 'TETUÁN',
 20: 'SAN BLAS-CANILLEJAS'}

In [1010]:
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].fillna(ds['accidentalidad']['cod_distrito'].map(distritoDict))

In [1011]:
get_info('accidentalidad')
# Get all info on accidentalidad

num_expediente               0
fecha                        0
hora                         0
localizacion                 0
numero                       6
cod_distrito                 6
distrito                     6
tipo_accidente               5
estado_meteorológico     20120
tipo_vehiculo              816
tipo_persona                 3
rango_edad                   0
sexo                         0
cod_lesividad                0
lesividad                    0
coordenada_x_utm             3
coordenada_y_utm             3
positiva_alcohol           718
positiva_droga          195001
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 31157
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   num_expediente        195610 non-null  object 
 1   fecha                 195610 non-null  object 
 2   hora                  195610 non-null  object 
 3   localizacion          195610

Por otro lado, positivo en droga tiene valor sólo si daba positivo, por lo que rellenar los valores faltantes con 0 es lo más lógico (siendo 0 negativo en droga). 

In [1012]:
ds['accidentalidad']['positiva_droga'] = ds['accidentalidad']['positiva_droga'].fillna(0)

Observemos primero los valores que faltan y que vamos a decidir rellenar muy probablemente de manera manual, ya que son menos de 10 valores por atributo que faltan. En concreto, las columnas referentes al distrito, a la localizacion (el número y las coordenadas), y el tipo de accidente y persona.

In [1013]:
ds['accidentalidad'][ds['accidentalidad'][['distrito', 'cod_distrito', 'tipo_accidente', 'tipo_persona', 'coordenada_x_utm', 'coordenada_y_utm']].isna().any(axis=1)]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
27635,2020S016821,23/11/2020,7:45:00,"AUTOV. M-23, 0 (0.8 ENTRADA)",NaN,NaN,NaN,Colisión lateral,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,"444975,42","4474103,079",N,0.0
27636,2020S016821,23/11/2020,7:45:00,"AUTOV. M-23, 0 (0.8 ENTRADA)",NaN,NaN,NaN,Colisión lateral,Despejado,Turismo,Conductor,De 50 a 54 años,Mujer,14.0,Sin asistencia sanitaria,"444975,42","4474103,079",N,0.0
32430,2020S019647,18/11/2020,10:57:00,"CALL. LOPE DE HARO, 8",8,6.0,TETUÁN,Colisión fronto-lateral,Despejado,Camión rígido,Conductor,De 40 a 44 años,Hombre,14.0,Sin asistencia sanitaria,NaN,NaN,N,0.0
32431,2020S019647,18/11/2020,10:57:00,"CALL. LOPE DE HARO, 8",8,6.0,TETUÁN,Colisión fronto-lateral,Despejado,Furgoneta,Conductor,De 45 a 49 años,Hombre,14.0,Sin asistencia sanitaria,NaN,NaN,N,0.0
25744,2021S015933,11/09/2021,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,NaN,NaN,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,14.0,Sin asistencia sanitaria,"437390,155","4476258,031",N,0.0
25745,2021S015933,11/09/2021,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,NaN,NaN,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 55 a 59 años,Hombre,14.0,Sin asistencia sanitaria,"437390,155","4476258,031",N,0.0
25746,2021S015933,11/09/2021,19:40:00,AUTOV. M-500 / AUTOV. M-30,NaN,NaN,NaN,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 55 a 59 años,Mujer,14.0,Sin asistencia sanitaria,"437390,155","4476258,031",N,0.0
36915,2021S022402,27/11/2021,2:10:00,PASEO. PRADO / PLAZA. CANOVAS DEL CASTILLO,36,1.0,CENTRO,NaN,NaN,Turismo,Conductor,De 55 a 59 años,Hombre,14.0,Sin asistencia sanitaria,"441066,084","4474148,543",N,0.0
36916,2021S022402,27/11/2021,2:10:00,PASEO. PRADO / PLAZA. CANOVAS DEL CASTILLO,36,1.0,CENTRO,NaN,NaN,Turismo,Pasajero,De 35 a 39 años,Hombre,14.0,Sin asistencia sanitaria,"441066,084","4474148,543",N,0.0
36917,2021S022402,27/11/2021,2:10:00,PASEO. PRADO / PLAZA. CANOVAS DEL CASTILLO,36,1.0,CENTRO,NaN,NaN,VMU eléctrico,Conductor,De 25 a 29 años,Hombre,2.0,Ingreso inferior o igual a 24 horas,"441066,084","4474148,543",S,0.0


In [1014]:
ds['accidentalidad'].at[10110, 'cod_distrito'] = 21
ds['accidentalidad'].at[27635, 'cod_distrito'] = 15
ds['accidentalidad'].at[27636, 'cod_distrito'] = 15
ds['accidentalidad'].at[25744, 'cod_distrito'] = 9
ds['accidentalidad'].at[25745, 'cod_distrito'] = 9
ds['accidentalidad'].at[25746, 'cod_distrito'] = 9
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].fillna(ds['accidentalidad']['cod_distrito'].map(distritoDict))

In [1015]:
ds['accidentalidad']['numero'] = ds['accidentalidad']['numero'].fillna('')

Para el tipo de persona, usamos rango de edad. Si la persona es menor de edad, no era conductor, y si el rango de edad es desconocido, el tipo de persona es conductor si hay una única incidencia.

In [1016]:
ds['accidentalidad'][(ds['accidentalidad']['tipo_persona'].isna()) & 
                              (ds['accidentalidad'].duplicated(subset=['fecha', 'hora', 'coordenada_x_utm', 'coordenada_y_utm'], keep=False))]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
37104,2021S022518,27/11/2021,22:25:00,"AVDA. HISPANIDAD, 34",34,21.0,BARAJAS,Alcance,Despejado,Turismo,NaN,Desconocido,Desconocido,14.0,Sin asistencia sanitaria,"451305,806","4479449,657",N,0.0
40495,2021S024425,21/12/2021,16:30:00,AVENIDA MANUEL FRAGA IRIBARNE NUMERO 2,35,21.0,BARAJAS,Alcance,Despejado,Turismo,NaN,Menor de 5 años,Hombre,14.0,Sin asistencia sanitaria,"448419,744","4481821,54",N,0.0
41575,2021S024983,30/12/2021,17:50:00,CALL. VALDETORRES DE JARAMA / CALL. ANGEL LUIS...,2,16.0,HORTALEZA,Colisión fronto-lateral,Despejado,Motocicleta hasta 125cc,NaN,Desconocido,Desconocido,14.0,Sin asistencia sanitaria,"445289,712","4480250,615",N,0.0


In [1017]:
ds['accidentalidad'].at[40495, 'tipo_persona'] = 'Pasajero'
ds['accidentalidad'].at[41575 , 'tipo_persona'] = 'Conductor'
ds['accidentalidad'].at[37104, 'tipo_persona'] = 'Conductor'

Para las coordenadas, las buscamos en internet y las rellenamos manualmente

In [1018]:
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S019647', ['coordenada_x_utm', 'coordenada_y_utm']] = [440444, 4478766]
ds['accidentalidad'].at[41231, 'coordenada_x_utm'] = 445058
ds['accidentalidad'].at[41231, 'coordenada_y_utm'] = 4475358
ds['accidentalidad'].at[26525, 'coordenada_x_utm'] = 440766
ds['accidentalidad'].at[26525, 'coordenada_y_utm'] = 4468560

Confirmamos que realmente no queda ningun NaN en las coordenadas

In [1019]:
df = ds['accidentalidad'][(ds['accidentalidad']['coordenada_x_utm'] == '#¡VALOR!') | (ds['accidentalidad']['coordenada_y_utm'] == '#¡VALOR!')]

Como sí hay, mostremos los numeros de expediente con la localizacion y el distrito, para poder buscar las coordenadas manualmente

In [1020]:
df[['localizacion', 'distrito', 'num_expediente', 'tipo_accidente']].drop_duplicates(subset='num_expediente')

,localizacion,distrito,num_expediente,tipo_accidente
29308,"AUTOV. M-30, +03200E",FUENCARRAL-EL PARDO,2019S026997,Alcance
17953,"CALL. POLVORANCA, 13",CARABANCHEL,2020S011111,Choque contra obstáculo fijo
20493,AVDA. ALBUFERA / AVDA. PABLO NERUDA,PUENTE DE VALLECAS,2020S012560,Colisión fronto-lateral
30817,RONDA. SUR / CALL. MARTOS,PUENTE DE VALLECAS,2021S018891,Alcance
37939,"CALL. REY FRANCISCO, 28",MONCLOA-ARAVACA,2021S022950,Colisión múltiple
39443,"AUTOV. M-23, KM 0,700",MORATALAZ,2021S023850,Colisión múltiple
41425,URB. MANZANA DE AZCA / CALL. AGUSTIN DE BETANC...,TETUÁN,2021S024915,Choque contra obstáculo fijo
41524,CALL. GENERAL RICARDOS / CALL. BATALLA DE TORR...,CARABANCHEL,2021S024959,Alcance
41614,"CALL. ARROYO DE LA MEDIA LEGUA, 72",MORATALAZ,2021S024999,Atropello a persona
802,"CALL. GENERAL RICARDOS, 15",CARABANCHEL,2022S000436,Caída


In [1021]:
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2019S026997', ['coordenada_x_utm', 'coordenada_y_utm']] = [440339, 4481526]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S011111', ['coordenada_x_utm', 'coordenada_y_utm']] = [436021, 4469424]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S012560', ['coordenada_x_utm', 'coordenada_y_utm']] = [445353, 4471146]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S018891', ['coordenada_x_utm', 'coordenada_y_utm']] = [443848, 4469603]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S022950', ['coordenada_x_utm', 'coordenada_y_utm']] = [439091, 4475312]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S023850', ['coordenada_x_utm', 'coordenada_y_utm']] = [444822, 4474145]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S024915', ['coordenada_x_utm', 'coordenada_y_utm']] = [441103, 4477883]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S024959', ['coordenada_x_utm', 'coordenada_y_utm']] = [437104, 4470605]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2021S024999', ['coordenada_x_utm', 'coordenada_y_utm']] = [444742, 4473996]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S000436', ['coordenada_x_utm', 'coordenada_y_utm']] = [439117, 4472055]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S036787', ['coordenada_x_utm', 'coordenada_y_utm']] = [442816, 4475162]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S037553', ['coordenada_x_utm', 'coordenada_y_utm']] = [444195, 4471576]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S039457', ['coordenada_x_utm', 'coordenada_y_utm']] = [438802, 4468600]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S040326', ['coordenada_x_utm', 'coordenada_y_utm']] = [443181, 4472205]
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2022S041278', ['coordenada_x_utm', 'coordenada_y_utm']] = [440884, 4477570]


In [1022]:
ds['accidentalidad']['tipo_accidente'] = ds['accidentalidad']['tipo_accidente'].fillna(ds['accidentalidad']['tipo_accidente'].mode()[0])

In [1023]:
tipos_accidente = ds['accidentalidad']['tipo_accidente'].unique()
for tipo in tipos_accidente:
    print(tipo)

Colisión lateral
Alcance
Choque contra obstáculo fijo
Colisión fronto-lateral
Caída
Colisión frontal
Otro
Atropello a persona
Colisión múltiple
Vuelco
Atropello a animal
Solo salida de la vía
Despeñamiento


Según la documentación de noviembre de 2023, redefinimos los tipos, siendo 
- Colisión doble: Accidente de tráfico ocurrido entre dos vehículos en movimiento, (colisión frontal, fronto lateral, lateral)
    - Aquí incluimos los antiguos tipos Colisión lateral, Colisión fronto-lateral, Colisión frontal
- Otras causas: Recoge los accidentes por atropello a animal, despeñamiento, salida de la vía, y otros
    - Incluimos los antiguos tipos Otro, Despeñamiento, Atropello a animal, y Solo salida de la vía
- Choque contra obstáculo o elemento de la vía: Accidente ocurrido entre un vehículo en movimiento con conductor y un objeto inmóvil que ocupa la vía o zona apartada de la misma, ya sea vehículo estacionado, árbol, farola, etc. 
    - Este no más es un cambio de nombre de Choque contra obstáculo fijo

Los tipos que se mantienen son:

- Alcance: Accidente que se produce cuando un vehículo circulando o detenido por las 
circunstancias del tráfico es golpeado en su parte posterior por otro vehículo.
- Caída: Se agrupan todas las caídas relacionadas con el desarrollo y las circunstancias del 
tráfico, (motocicleta, ciclomotor, bicicleta, viajero bus, etc.)
- Atropello a persona: Accidente ocurrido ente un vehículo y un peatón que ocupa la calzada 
o que transita por aceras, refugios, paseos o zonas de la vía pública no destinada a la
circulación de vehículos
- Vuelco: Accidente sufrido por un vehículo con más de dos ruedas y que por alguna 
circunstancia sus neumáticos pierden el contacto con la calzada quedando apoyado sobre 
un costado o sobre el techo.
- Colisión múltiple: Accidente de tráfico ocurrido entre más de dos vehículos en movimiento.


In [1024]:
ds['accidentalidad']['tipo_accidente'] = pd.Categorical(ds['accidentalidad']['tipo_accidente'])
ds['accidentalidad']['cod_accidente'] = ds['accidentalidad']['tipo_accidente'].cat.codes
accidenteDict = ds['accidentalidad'].set_index('cod_accidente')['tipo_accidente'].to_dict()
accidenteDict

{7: 'Colisión lateral',
 0: 'Alcance',
 4: 'Choque contra obstáculo fijo',
 6: 'Colisión fronto-lateral',
 3: 'Caída',
 5: 'Colisión frontal',
 10: 'Otro',
 2: 'Atropello a persona',
 8: 'Colisión múltiple',
 12: 'Vuelco',
 1: 'Atropello a animal',
 11: 'Solo salida de la vía',
 9: 'Despeñamiento'}

In [1025]:
ds['accidentalidad']['tipo_accidente'] = ds['accidentalidad']['tipo_accidente'].cat.add_categories(['Colisión doble', 'Otras causas', 'Choque contra obstáculo o elemento de la vía'])

colisionDoble = ['Colisión lateral', 'Colisión fronto-lateral', 'Colisión frontal']
ds['accidentalidad'].loc[ds['accidentalidad']['tipo_accidente'].isin(colisionDoble), 'tipo_accidente'] = 'Colisión doble'

otrasCausas = ['Otro', 'Despeñamiento', 'Atropello a animal', 'Solo salida de la vía']
ds['accidentalidad'].loc[ds['accidentalidad']['tipo_accidente'].isin(otrasCausas), 'tipo_accidente'] = 'Otras causas'

obstaculoFijo = ['Choque contra obstáculo fijo']
ds['accidentalidad'].loc[ds['accidentalidad']['tipo_accidente'].isin(otrasCausas), 'tipo_accidente'] = 'Choque contra obstáculo o elemento de la vía'


In [1026]:
get_info('accidentalidad')

num_expediente              0
fecha                       0
hora                        0
localizacion                0
numero                      0
cod_distrito                0
distrito                    0
tipo_accidente              0
estado_meteorológico    20120
tipo_vehiculo             816
tipo_persona                0
rango_edad                  0
sexo                        0
cod_lesividad               0
lesividad                   0
coordenada_x_utm            0
coordenada_y_utm            0
positiva_alcohol          718
positiva_droga              0
cod_accidente               0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 31157
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   num_expediente        195610 non-null  object  
 1   fecha                 195610 non-null  object  
 2   hora                  195610 non-null  object  
 3   localizacion

In [1027]:
ds['accidentalidad']['cod_distrito'] = ds['accidentalidad']['cod_distrito'].astype('int64')
ds['accidentalidad']['cod_lesividad'] = ds['accidentalidad']['cod_lesividad'].astype('int64')
ds['accidentalidad']['tipo_accidente'] = ds['accidentalidad']['tipo_accidente'].astype('category')
ds['accidentalidad']['lesividad'] = ds['accidentalidad']['lesividad'].astype('category')
ds['accidentalidad']['sexo'] = ds['accidentalidad']['sexo'].astype('category')
ds['accidentalidad']['tipo_persona'] = ds['accidentalidad']['tipo_persona'].astype('category')
ds['accidentalidad']['distrito'] = ds['accidentalidad']['distrito'].astype('category')
ds['accidentalidad']['rango_edad'] = ds['accidentalidad']['rango_edad'].astype('category')
ds['accidentalidad']['positiva_droga'] = ds['accidentalidad']['positiva_droga'].astype('category')

ds['accidentalidad']['coordenada_x_utm'] = ds['accidentalidad']['coordenada_x_utm'].astype('str').str.replace(',', '.')
ds['accidentalidad']['coordenada_y_utm'] = ds['accidentalidad']['coordenada_y_utm'].astype('str').str.replace(',', '.')

ds['accidentalidad']['coordenada_x_utm'] = ds['accidentalidad']['coordenada_x_utm'].astype('float64')
ds['accidentalidad']['coordenada_y_utm'] = ds['accidentalidad']['coordenada_y_utm'].astype('float64')

in_proj = Proj(init='epsg:25830')  # EPSG: 25830 = 30N
out_proj = Proj(init='epsg:4326')  # sistema WGS84 delatlong

ds['accidentalidad']['longitud'], ds['accidentalidad']['latitud'] = transform(in_proj, out_proj, ds['accidentalidad']['coordenada_x_utm'].values, ds['accidentalidad']['coordenada_y_utm'].values)

ds['accidentalidad']['hora'] = ds['accidentalidad']['hora'].str.split(':')
ds['accidentalidad']['hora'] = ds['accidentalidad']['hora'].apply(lambda x: [i.zfill(2) for i in x])
ds['accidentalidad']['hora'] = ds['accidentalidad']['hora'].apply(lambda x: ':'.join(x))

ds['accidentalidad']['datetime'] = pd.to_datetime(ds['accidentalidad']['fecha'] + ' ' + ds['accidentalidad']['hora'], format='%d/%m/%Y %H:%M:%S')

ds['accidentalidad']['hora'] = ds['accidentalidad']['datetime'].dt.time
ds['accidentalidad']['fecha'] = ds['accidentalidad']['datetime'].dt.date

ds['accidentalidad']['localizacion'] = ds['accidentalidad']['localizacion'].fillna('') + ' ' + ds['accidentalidad']['numero'].fillna('')
ds['accidentalidad'].drop('numero', axis=1, inplace=True)

c:\Users\paula\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\paula\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\paula\AppData\Local\Temp\ipykernel_21672\1880648825.py:20: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/got

In [1028]:
get_info('accidentalidad')

num_expediente              0
fecha                       0
hora                        0
localizacion                0
cod_distrito                0
distrito                    0
tipo_accidente              0
estado_meteorológico    20120
tipo_vehiculo             816
tipo_persona                0
rango_edad                  0
sexo                        0
cod_lesividad               0
lesividad                   0
coordenada_x_utm            0
coordenada_y_utm            0
positiva_alcohol          718
positiva_droga              0
cod_accidente               0
longitud                    0
latitud                     0
datetime                    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 31157
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   num_expediente        195610 non-null  object        
 1   fecha                 195610 non-null  o

Por último, donde queda dilema es en positivo alcohol, tipo de vehículo y estado meteorológico. En esta situación, lo más apropiado es ver si, relacionando estos atributos con algún otro, es más probable que los atributos valgan uno u otro valor.

- Estado meteorológico. Hay 7 valores posibles: despejado, lluvia débil, lluvia intensa, granizando, nevando, nublado, se desconoce. Aquí, por lo tanto, hay 3 vías de actuación:
    - Rellenar con "se desconoce", i.e.: ser fieles a lo que se sabe, reducir la proporción de datos artificiales (hay un 11% de datos faltantes), solución sencilla.
    - Rellenar con el valor más frecuente: despejado (representa el 75% de los datos), i.e.: solución con datos artificiales más sencilla.
    - Rellenar con valores aleatorios según la proporción en la que aparecen los datos, i.e.: el 75% de los datos faltantes se rellenan arbitrariamente con "Despejado".
    - Rellenar con el valor que ya hubiera ese día en esa franja horaria.

    Lo que mejor preserva los datos es, rellenar con "se desconoce", pues la variable existe previamente.

In [1029]:
ds['accidentalidad'].set_index('datetime', inplace=True)

# Create a new column for the 12-hour intervals
ds['accidentalidad']['time_group'] = ds['accidentalidad'].index.to_series().dt.floor('12H')

# Group by the new column and forward fill within each group for 'estado_meteorológico' column
ds['accidentalidad']['estado_meteorológico'] = ds['accidentalidad'].groupby('time_group')['estado_meteorológico'].ffill()

# Fill any remaining NaNs with the next valid observation
ds['accidentalidad']['estado_meteorológico'] = ds['accidentalidad'].groupby('time_group')['estado_meteorológico'].bfill()

# Drop the 'time_group' column
ds['accidentalidad'].drop(columns='time_group', inplace=True)

# Infer the best data types for each column
ds['accidentalidad'] = ds['accidentalidad'].infer_objects()

# Reset the index
ds['accidentalidad'].reset_index(inplace=True)

C:\Users\paula\AppData\Local\Temp\ipykernel_21672\2173949171.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ds['accidentalidad']['time_group'] = ds['accidentalidad'].index.to_series().dt.floor('12H')


In [1030]:
get_info('accidentalidad')

datetime                  0
num_expediente            0
fecha                     0
hora                      0
localizacion              0
cod_distrito              0
distrito                  0
tipo_accidente            0
estado_meteorológico      3
tipo_vehiculo           816
tipo_persona              0
rango_edad                0
sexo                      0
cod_lesividad             0
lesividad                 0
coordenada_x_utm          0
coordenada_y_utm          0
positiva_alcohol        718
positiva_droga            0
cod_accidente             0
longitud                  0
latitud                   0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195610 entries, 0 to 195609
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   datetime              195610 non-null  datetime64[ns]
 1   num_expediente        195610 non-null  object        
 2   fecha              

Como siguen quedando 3 valores NaN, los rellenamos manualmente. En caso de que fueran una cantidad significativa, repetiríamos el proceso anterior, aumentando la ventana a 24h (lo cual es equivalente a lo que vamos a hacer manualmente).

In [1031]:
nan_dates = ds['accidentalidad'][ds['accidentalidad']['estado_meteorológico'].isna()]['fecha']
mask = ds['accidentalidad']['fecha'].isin(nan_dates)
estados = ds['accidentalidad'][mask].sort_values(by='datetime')

In [1032]:
# fillna of ds['accidentalidad']['estado_meteorológico'] where num_expediente is 2020S006036 with Despejado
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S006036', 'estado_meteorológico'] = 'Despejado'
ds['accidentalidad'].loc[ds['accidentalidad']['num_expediente'] == '2020S006030', 'estado_meteorológico'] = 'Despejado'

Convert 'hora' to datetime and extract the hour
ds['accidentalidad']['hora'] = pd.to_datetime(ds['accidentalidad']['hora'], format='%H:%M:%S')
ds['accidentalidad']['HOUR'] = ds['accidentalidad']['hora'].dt.hour

Categorize hours into different time periods
bins = [-1, 6, 12, 18, 24]
labels = ['Noche', 'Mañana', 'Media-Tarde', 'Tarde-Noche']
ds['accidentalidad']['TIME_OF_DAY'] = pd.cut(ds['accidentalidad']['HOUR'], bins=bins, labels=labels, include_lowest=True)

In [1033]:
ds['accidentalidad']['tipo_vehiculo'].unique()

array(['Motocicleta > 125cc', 'Turismo', 'Furgoneta', 'Autobús',
       'Ciclomotor', 'Motocicleta hasta 125cc', 'Todo terreno',
       'Bicicleta', 'Camión rígido', 'Maquinaria de obras',
       'Tractocamión', nan, 'Cuadriciclo no ligero',
       'Vehículo articulado', 'Autobús articulado',
       'Otros vehículos con motor', 'Autocaravana', 'Patinete', 'Ciclo',
       'Cuadriciclo ligero', 'VMU eléctrico', 'Semiremolque',
       'Microbús <= 17 plazas', 'Sin especificar', 'Autobus EMT',
       'Remolque', 'Tranvía', 'Caravana', 'Camión de bomberos',
       'Otros vehículos sin motor', 'Bicicleta EPAC (pedaleo asistido)',
       'Moto de tres ruedas > 125cc', 'Tren/metro', 'Ambulancia SAMUR',
       'Moto de tres ruedas hasta 125cc',
       'Ciclomotor de dos ruedas L1e-B', 'Maquinaria agrícola',
       'Autobús articulado EMT', 'Ciclomotor de tres ruedas',
       'Ciclo de motor L1e-A', 'Patinete no eléctrico'], dtype=object)

- Tipo de vehículo. Hay 34 valores posibles: Ambulancia SAMUR, autobús EMT, autobús, autobús articulado, autobús articulado EMT, autocaravana, bicicleta, bicicleta EPAC (pedaleo asistido), camión de bomberos, camión rígido, ciclo, ciclomotor, ciclomotor de dos ruedas L1e-B, cuadriciclo ligero, cuadriciclo no ligero, furgoneta, maquinaria de obras, microbús <= 17 plazas, moto de tres ruedas > 125cc, moto de tres ruedas hasta 125cc, motocicleta > 125cc, motocicleta hasta 125cc, otros vehículos con motor, otros vehículos sin motor, patinete no eléctrico, remolque, semirremolque, sin especificar, todo terreno, tractocamión, tren/metro, turismo (68%), VMU eléctrico, vehículo articulado. En esta variable hay 0.6% de valores faltantes, lo cual no es significativo, i.e.: la sustitución que elijamos tendrá menos repercusión en el estudio final. Aquí, por lo tanto, hay 2 vías de actuación:
    - Rellenar con "sin especificar", i.e.: solución sencilla y descriptiva pero que puede dar lugar a interpretaciones erróneas, pues puede haber sido otro tipo de vehículo que no se había registrado.
    - Rellenar con el valor más probable según otro atributo (por ejemplo, código de lesividad).
    
    La solución más apropiada es rellenar con el valor más probable según código de lesividad, por lo que vamos a ver primero cómo se relacionan ambos atributos y después rellenaremos los valores faltantes con el valor más probable.

In [1034]:
import scipy.stats as stats
contingency_table = pd.crosstab(ds['accidentalidad']['tipo_vehiculo'], ds['accidentalidad']['cod_lesividad'])
chi2, p, _, _ = stats.chi2_contingency(contingency_table)
if (p < 0.05):
    print('Tipo de vehículo y lesividad están relacionados')

Tipo de vehículo y lesividad están relacionados


Por lo tanto, es coherente rellenar el tipo de vehiculo según el grado de lesividad

In [1035]:
ds['accidentalidad']['tipo_vehiculo'] = ds['accidentalidad']['tipo_vehiculo'].fillna(ds['accidentalidad'].groupby('cod_lesividad')['tipo_vehiculo'].transform(lambda x:x.mode().iat[0]))

Para rellenar los datos de positivo en alcohol, usaremos lo más habitual según grupo de edad, sexo y lesividad de la persona

In [1036]:
ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad'].groupby(['rango_edad','sexo','lesividad'])['positiva_alcohol'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))

C:\Users\paula\AppData\Local\Temp\ipykernel_21672\1668006814.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad'].groupby(['rango_edad','sexo','lesividad'])['positiva_alcohol'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))


In [1037]:
ds['accidentalidad'] = ds['accidentalidad'][ds['accidentalidad'].distrito != '13.0']

In [1038]:
ds['accidentalidad'] = ds['accidentalidad'].loc[ds['accidentalidad']['tipo_accidente'] != 13.0]

In [1039]:
get_info('accidentalidad')

datetime                0
num_expediente          0
fecha                   0
hora                    0
localizacion            0
cod_distrito            0
distrito                0
tipo_accidente          0
estado_meteorológico    0
tipo_vehiculo           0
tipo_persona            0
rango_edad              0
sexo                    0
cod_lesividad           0
lesividad               0
coordenada_x_utm        0
coordenada_y_utm        0
positiva_alcohol        0
positiva_droga          0
cod_accidente           0
longitud                0
latitud                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 195609
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   datetime              195610 non-null  datetime64[ns]
 1   num_expediente        195610 non-null  object        
 2   fecha                 195610 non-null  object        
 3   hora     

In [1040]:
ds['accidentalidad']['tipo_vehiculo'] = ds['accidentalidad']['tipo_vehiculo'].astype('category')
ds['accidentalidad']['positiva_alcohol'] = ds['accidentalidad']['positiva_alcohol'].astype('category')
ds['accidentalidad']['estado_meteorológico'] = ds['accidentalidad']['estado_meteorológico'].astype('category')

In [1041]:
get_info('accidentalidad')

datetime                0
num_expediente          0
fecha                   0
hora                    0
localizacion            0
cod_distrito            0
distrito                0
tipo_accidente          0
estado_meteorológico    0
tipo_vehiculo           0
tipo_persona            0
rango_edad              0
sexo                    0
cod_lesividad           0
lesividad               0
coordenada_x_utm        0
coordenada_y_utm        0
positiva_alcohol        0
positiva_droga          0
cod_accidente           0
longitud                0
latitud                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 195609
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   datetime              195610 non-null  datetime64[ns]
 1   num_expediente        195610 non-null  object        
 2   fecha                 195610 non-null  object        
 3   hora     

Con esto, queda limpio el dataset de accidentalidad

In [1055]:
ds['accidentalidad'].to_csv('limpios/accidentalidad.csv', index=False)

#### Aforo del tráfico

In [1053]:
get_info('estaciones')
ds['estaciones']

fdia     0
fest     0
fsen     0
hor1     0
hor2     0
hor3     0
hor4     0
hor5     0
hor6     0
hor7     0
hor8     0
hor9     0
hor10    0
hor11    0
hor12    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 251104 entries, 0 to 7079
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   fdia    251104 non-null  datetime64[ns]
 1   fest    251104 non-null  object        
 2   fsen    251104 non-null  object        
 3   hor1    251104 non-null  int64         
 4   hor2    251104 non-null  int64         
 5   hor3    251104 non-null  int64         
 6   hor4    251104 non-null  int64         
 7   hor5    251104 non-null  int64         
 8   hor6    251104 non-null  int64         
 9   hor7    251104 non-null  int64         
 10  hor8    251104 non-null  int64         
 11  hor9    251104 non-null  int64         
 12  hor10   251104 non-null  int64         
 13  hor11   251104 non-null  int64        

,fdia,fest,fsen,hor1,hor2,hor3,hor4,hor5,hor6,hor7,hor8,hor9,hor10,hor11,hor12
0,2021-04-01,ES01,1-,38,32,25,17,46,165,176,218,331,467,662,811
1,2021-04-01,ES01,1=,944,856,577,720,886,952,1067,1100,941,1029,724,86
2,2021-04-01,ES01,2-,62,27,24,37,31,130,126,232,330,606,771,978
3,2021-04-01,ES01,2=,1084,912,608,824,1019,1209,1089,949,724,685,597,147
4,2021-04-01,ES02,1-,221,80,55,44,53,48,80,138,184,260,335,430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,2023-09-30,ES59,2=,174,179,141,169,134,176,214,148,179,93,181,68
7076,2023-09-30,ES60,1-,121,96,50,58,70,94,119,195,325,432,438,507
7077,2023-09-30,ES60,1=,448,352,250,239,292,315,326,365,276,198,163,190
7078,2023-09-30,ES60,2-,127,101,62,56,36,64,87,146,239,367,380,463


In [1048]:
ds['estaciones']['fdia'] = pd.to_datetime(ds['estaciones']['fdia'], format='mixed', dayfirst=True)
# change dtype of attributes hor1, hor2, hor3, ..., hor12 to int with fstring
for i in range(1, 13):
    ds['estaciones'][f'hor{i}'] = ds['estaciones'][f'hor{i}'].astype('int64')


Este dataset no tiene datos faltantes. Apreciamos que FSEN, aparte de indicar el sentido del tráfico, también indica si son las primeras 12h del día (1- y 2-), o las últimas (1=, 2=). Hacemos 12 nuevos atributos para suplir esto en un formato 24h que es más cómodo.

In [1052]:
get_info('estaciones')

fdia     0
fest     0
fsen     0
hor1     0
hor2     0
hor3     0
hor4     0
hor5     0
hor6     0
hor7     0
hor8     0
hor9     0
hor10    0
hor11    0
hor12    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 251104 entries, 0 to 7079
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   fdia    251104 non-null  datetime64[ns]
 1   fest    251104 non-null  object        
 2   fsen    251104 non-null  object        
 3   hor1    251104 non-null  int64         
 4   hor2    251104 non-null  int64         
 5   hor3    251104 non-null  int64         
 6   hor4    251104 non-null  int64         
 7   hor5    251104 non-null  int64         
 8   hor6    251104 non-null  int64         
 9   hor7    251104 non-null  int64         
 10  hor8    251104 non-null  int64         
 11  hor9    251104 non-null  int64         
 12  hor10   251104 non-null  int64         
 13  hor11   251104 non-null  int64        

In [1056]:
df = ds['estaciones']

df[['direccion', 'ampm']] = df['fsen'].str.extract(r'(\d)([=-])')
df.drop(['fsen'], axis=1, inplace=True)

index = ['fdia', 'fest', 'direccion', 'ampm', 'hor1', 'hor2', 'hor3', 'hor4', 'hor5', 'hor6', 'hor7', 'hor8', 'hor9', 'hor10', 'hor11', 'hor12', 'hor13', 'hor14', 'hor15', 'hor16', 'hor17', 'hor18', 'hor19', 'hor20', 'hor21', 'hor22', 'hor23', 'hor24']

df.reindex(columns=index)

,fdia,fest,direccion,ampm,hor1,hor2,hor3,hor4,hor5,hor6,...,hor15,hor16,hor17,hor18,hor19,hor20,hor21,hor22,hor23,hor24
0,2021-04-01,ES01,1,-,38,32,25,17,46,165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-01,ES01,1,=,944,856,577,720,886,952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01,ES01,2,-,62,27,24,37,31,130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-01,ES01,2,=,1084,912,608,824,1019,1209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-01,ES02,1,-,221,80,55,44,53,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,2023-09-30,ES59,2,=,174,179,141,169,134,176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7076,2023-09-30,ES60,1,-,121,96,50,58,70,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7077,2023-09-30,ES60,1,=,448,352,250,239,292,315,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7078,2023-09-30,ES60,2,-,127,101,62,56,36,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def group_fn(dff: pd.DataFrame):
    # dff contiene 2 rows de am y pm
    am = dff.loc[dff['FSEN_AMPM'] == "-"].iloc[0]
    pm = dff.loc[dff['FSEN_AMPM'] == "="].iloc[0]
    par = am[['hor1', 'hor2', 'hor3', 'hor4', 'hor5', 'hor6', 'hor7', 'hor8', 'hor9', 'hor10', 'hor11', 'hor12']].copy()
    
    par['hor13'] = pm['hor1']
    par['hor14'] = pm['hor2']
    par['hor15'] = pm['hor3']
    par['hor16'] = pm['hor4']
    par['hor17'] = pm['hor5']
    par['hor18'] = pm['hor6']
    par['hor19'] = pm['hor7']
    par['hor20'] = pm['hor8']
    par['hor21'] = pm['hor9']
    par['hor22'] = pm['hor10']
    par['hor23'] = pm['hor11']
    par['hor24'] = pm['hor12']

    return par

In [ ]:
ds['datasets/estaciones'] = df.groupby(['FEST', 'fdia', 'FSEN_DIRECTION']).apply(group_fn)
ds['datasets/estaciones']

In [ ]:
get_info('datasets/estaciones')

In [ ]:
dataframe_summary('datasets/estaciones')

In [ ]:
get_info('direcciones')

Como podemos observar, faltan datos de las columnas VIA_SQC (secuencia de la denominación), con 371584 NaNs; VIA_PAR (partícula de la denominación), con 17997; CALIFICADOR (del número de la policía), con 322826; FECHA_DE_BAJA (de la dirección), con 211657; y TIPO_NDP (tipo de número: portal, garaje, fachada, etc.) con 16248.

In [ ]:
ds['direcciones']

La secuencia de denominación (VIA_SQC) es solo NaNs, por lo que lo más lógico es eliminar la columna directamente

In [ ]:
ds['direcciones'] = ds['direcciones'].drop('VIA_SQC', axis=1)

Ahora, pasamos a la partícula de denominación (VIA_PAR), hay 7 valores distintos de entre la lista posible de 20 (DE, EL, A, DEL, LA, AL, DE LA, LAS, A LA, DE LAS, LO, A LAS, DE LO, LOS, A LO, DE LO, DE LOS, POR EL, A LOS, POR LA), de los cuales el primero es A LA y el último es DEL. Por lo tanto, parece que no se ha rellenado con Null o espacio vacío aquellas direcciones en las que no hay partícula:

In [ ]:
ds['direcciones']['VIA_PAR'] = ds['direcciones']['VIA_PAR'].fillna(' ')

Fecha de baja, si no hay valor, es que está aún de alta, así que también sustituimos por un valor que represente que sigue dado de alta:

In [ ]:
ds['direcciones']['FECHA_DE_BAJA'] = ds['direcciones']['FECHA_DE_BAJA'].fillna('--/--/----')

Para Tipo de número de policía, rellenaremos con el dato más habitual: PORTAL

In [ ]:
tipoNDP = ds['direcciones']['TIPO_NDP'].mode().iat[0]
ds['direcciones']['TIPO_NDP'] = ds['direcciones']['TIPO_NDP'].fillna(tipoNDP)

Para Calificador, como no sabemos realmente cuál es el criterio y la mayoría de datos son faltantes (322826 faltan de 371584), lo más prudente es eliminar la columna directamente

In [ ]:
ds['direcciones'] = ds['direcciones'].drop('CALIFICADOR', axis=1)

In [ ]:
get_info('direcciones')

Efectivamente no quedan Missing Values. Pasamos a las direcciones vigentes:

In [ ]:
get_info('DireccionesVigentes_20231004.csv')

Faltan valores en Vía Par (3381) y en Calificador (178838), los sustituimos con el mismo criterio que en el dataset anterior:

In [ ]:
ds['DireccionesVigentes_20231004.csv']['VIA_PAR'] = ds['DireccionesVigentes_20231004.csv']['VIA_PAR'].fillna(' ')

In [ ]:
ds['DireccionesVigentes_20231004.csv'] = ds['DireccionesVigentes_20231004.csv'].drop('CALIFICADOR', axis=1)

In [ ]:
get_info('DireccionesVigentes_20231004.csv')

Pasamos al dataset estrella: radares.

In [ ]:
get_info('radares.csv')
ds['radares.csv']

Carretera o vial, PK, sentido y tipo tienen muy poca frecuencia de NaNs, por lo que solo sustituimos por los valores más frecuentes

In [ ]:
ds['radares.csv'] = ds['radares.csv'].drop('X_(WGS84)', axis=1)
ds['radares.csv'] = ds['radares.csv'].drop('Y_(WGS84)', axis=1)

In [ ]:
get_info('radares.csv')

In [ ]:
# Function to manually convert the provided format to decimal degrees
def manual_conversion(value):
    try:
        # Remove dots and convert to float
        value = float(value.replace('.', ''))
        # Divide by 10^6 to get approximate decimal degrees
        value /= 10**6
        return value
    except ValueError:
        return None

# Convert 'LATITUD' and 'LONGITUD' to decimal degrees
ds['radares.csv']['LATITUD'] = ds['radares.csv']['LATITUD'].apply(manual_conversion)
ds['radares.csv']['LONGITUD'] = ds['radares.csv']['LONGITUD'].apply(manual_conversion)

ds['radares.csv']

Rellenar los datos faltantes de accidentalidad en tipo persona y tipo vehículo a través de correlación entre ellos y código lesividad

In [ ]:
modo_tipo_persona = ds['accidentalidad'].groupby('cod_lesividad')['tipo_persona'].agg(lambda x: x.mode().iloc[0])
modo_tipo_vehiculo = ds['accidentalidad'].groupby('cod_lesividad')['tipo_vehiculo'].agg(lambda x: x.mode().iloc[0])
ds['accidentalidad']['tipo_persona'].fillna(ds['accidentalidad']['cod_lesividad'].map(modo_tipo_persona), inplace=True)
ds['accidentalidad']['tipo_vehiculo'].fillna(ds['accidentalidad']['cod_lesividad'].map(modo_tipo_vehiculo), inplace=True)
get_info('accidentalidad')


Añadir colummna de tipo de díaa en accidentalidad para saber el tipo de día (laboral , festivo)

In [ ]:
ds['accidentalidad']['FECHA'] = pd.to_datetime(ds['accidentalidad']['FECHA'])
ds['accidentalidad']['TIPO_DIA'] = ds['accidentalidad']['FECHA'].dt.day_name()
tipo_dia_mapping = {
    'Monday': 'Laboral',
    'Tuesday': 'Laboral',
    'Wednesday': 'Laboral',
    'Thursday': 'Laboral',
    'Friday': 'Laboral',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
ds['accidentalidad']['TIPO_DIA'] = ds['accidentalidad']['TIPO_DIA'].map(tipo_dia_mapping)
print(ds['accidentalidad']['TIPO_DIA'].value_counts())
